In [16]:
import re
import pandas as pd

def norm_name(name: str) -> str:
    if pd.isna(name): return ""
    s = str(name).lower()
    s = re.sub(r"[^a-z\s]", "", s)   # drop punctuation
    s = (s.replace(" jr","")
           .replace(" ii","")
           .replace(" iii","")
           .replace(" iv",""))
    return re.sub(r"\s+", " ", s).strip()


In [17]:
drafts = pd.read_csv("cleaned_draft_history.csv")
adps   = pd.read_csv("data/2016-2024_slim/fantasypros_adp_slim.csv")
fin    = pd.read_csv("data/2016-2024_slim/fantasypros_leaders_slim.csv")

# normalize names
for df in [drafts, adps, fin]:
    df["player_norm"] = df["Player"].map(norm_name)

In [6]:
import pandas as pd, re

# ---------- Load ----------
drafts = pd.read_csv("cleaned_draft_history.csv")
adps   = pd.read_csv("data/2016-2024_slim/fantasypros_adp_slim.csv")          # Year, Player, POS, Rank
fin    = pd.read_csv("data/2016-2024_slim/fantasypros_leaders_slim.csv")      # #, Player, Pos, Year, AVG, TTL

# ---------- Normalization helpers ----------
def norm_name(x: str) -> str:
    if pd.isna(x): return ""
    s = str(x).lower()
    # keep letters/digits/spaces/slash/apostrophe; normalize spaces
    s = re.sub(r"[^a-z0-9\s/']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Canonicalize to FantasyPros naming
ALIASES_RAW = {
    # Player renames / suffix differences (FP side canonical on the right)
    "Aaron Jones": "Aaron Jones Sr.",
    "Kyle Pitts": "Kyle Pitts Sr.",
    "Deboo Samuel": "Deebo Samuel Sr.",
    "Deebo Samuel": "Deebo Samuel Sr.",
    "Robby Anderson": "Robbie Chosen",
    "Robbie Anderson": "Robbie Chosen",
    "Chosen Anderson": "Robbie Chosen",
    "Will Fuller V": "William Fuller V",
    "Nyheim Hines": "Nyheim Miller-Hines",
    "Anthony Richardson": "Anthony Richardson Sr.",
    "Le'Veon Bell": "Le' Veon Bell",  # hedge spacing/punct variants
    "Todd Gurley II": "Todd Gurley II",
    "Melvin Gordon III": "Melvin Gordon III",
    "Larry Fitzgerald": "Larry Fitzgerald",
    # Common “II” gaps you surfaced
    "Patrick Mahomes": "Patrick Mahomes II",
    "Allen Robinson": "Allen Robinson II",
    "Allen Robinson ii": "Allen Robinson II",
    "Ronald Jones": "Ronald Jones II",
    "Ronald Jones ii": "Ronald Jones II",
    "Aj Dillon": "AJ Dillon",
    "A. J. Dillon": "AJ Dillon",

    # DST mascot → FP full city names (covers your unmatched list)
    "Steelers D/ST": "Pittsburgh Steelers D/ST",
    "Ravens D/ST": "Baltimore Ravens D/ST",
    "Cowboys D/ST": "Dallas Cowboys D/ST",
    "49ers D/ST": "San Francisco 49ers D/ST",
    "Bills D/ST": "Buffalo Bills D/ST",
    "Broncos D/ST": "Denver Broncos D/ST",
    "Saints D/ST": "New Orleans Saints D/ST",
    "Dolphins D/ST": "Miami Dolphins D/ST",
    "Rams D/ST": "Los Angeles Rams D/ST",
    "Jaguars D/ST": "Jacksonville Jaguars D/ST",
    "Chargers D/ST": "Los Angeles Chargers D/ST",
    "Patriots D/ST": "New England Patriots D/ST",
    "Buccaneers D/ST": "Tampa Bay Buccaneers D/ST",
    "Vikings D/ST": "Minnesota Vikings D/ST",
    "Texans D/ST": "Houston Texans D/ST",
    "Bears D/ST": "Chicago Bears D/ST",
    "Colts D/ST": "Indianapolis Colts D/ST",
    "Jets D/ST": "New York Jets D/ST",
    "Chiefs D/ST": "Kansas City Chiefs D/ST",
    "Browns D/ST": "Cleveland Browns D/ST",
    "Eagles D/ST": "Philadelphia Eagles D/ST",
    "Giants D/ST": "New York Giants D/ST",
    "Packers D/ST": "Green Bay Packers D/ST",
    "Bengals D/ST": "Cincinnati Bengals D/ST",
    "Commanders D/ST": "Washington Commanders D/ST",
    "Falcons D/ST": "Atlanta Falcons D/ST",
}

# normalize alias dict once
ALIASES = {norm_name(k): norm_name(v) for k, v in ALIASES_RAW.items()}

def apply_alias(n: str) -> str:
    n2 = norm_name(n)
    return ALIASES.get(n2, n2)

# Team code → FP full DST name
DST_FULL = {
    "ARI":"Arizona Cardinals D/ST","ATL":"Atlanta Falcons D/ST","BAL":"Baltimore Ravens D/ST",
    "BUF":"Buffalo Bills D/ST","CAR":"Carolina Panthers D/ST","CHI":"Chicago Bears D/ST",
    "CIN":"Cincinnati Bengals D/ST","CLE":"Cleveland Browns D/ST","DAL":"Dallas Cowboys D/ST",
    "DEN":"Denver Broncos D/ST","DET":"Detroit Lions D/ST","GB":"Green Bay Packers D/ST","GNB":"Green Bay Packers D/ST",
    "HOU":"Houston Texans D/ST","IND":"Indianapolis Colts D/ST","JAX":"Jacksonville Jaguars D/ST","JAC":"Jacksonville Jaguars D/ST",
    "KC":"Kansas City Chiefs D/ST","LAC":"Los Angeles Chargers D/ST","LAR":"Los Angeles Rams D/ST",
    "LV":"Las Vegas Raiders D/ST","LVR":"Las Vegas Raiders D/ST","MIA":"Miami Dolphins D/ST",
    "MIN":"Minnesota Vikings D/ST","NE":"New England Patriots D/ST","NO":"New Orleans Saints D/ST","NOR":"New Orleans Saints D/ST",
    "NYG":"New York Giants D/ST","NYJ":"New York Jets D/ST","PHI":"Philadelphia Eagles D/ST",
    "PIT":"Pittsburgh Steelers D/ST","SEA":"Seattle Seahawks D/ST","SF":"San Francisco 49ers D/ST","SFO":"San Francisco 49ers D/ST",
    "TB":"Tampa Bay Buccaneers D/ST","TEN":"Tennessee Titans D/ST","WAS":"Washington Commanders D/ST","WSH":"Washington Commanders D/ST",
    # legacy codes
    "OAK":"Oakland Raiders D/ST","SD":"San Diego Chargers D/ST","STL":"St. Louis Rams D/ST","LA":"Los Angeles Rams D/ST",
}

# ---------- Draft parsing ----------
pat = re.compile(r"^(?P<name>.+?)\s+(?P<team>[A-Za-z]{2,4})\s*,\s*(?P<pos>QB|RB|WR|TE|K|DST)$", re.I)

def parse_draft_player(s: str):
    s = str(s).strip()
    m = pat.match(s)
    if m:
        return m.group("name").strip(), m.group("team").upper(), m.group("pos").upper()
    if "," in s:
        left, pos = s.rsplit(",", 1)
        parts = left.rsplit(" ", 1)
        if len(parts) == 2:
            return parts[0].strip(), parts[1].upper(), pos.strip().upper()
    return s, "", ""

# build fields
ntp = drafts["Player"].map(parse_draft_player)
drafts["player_name_raw"] = ntp.map(lambda t: t[0])
drafts["nfl_team_code"]   = ntp.map(lambda t: t[1])
drafts["pos"]             = ntp.map(lambda t: t[2])

# tag blanks (keeping them so your row count stays unchanged)
drafts["is_blank"] = (
    drafts["player_name_raw"].isna() |
    drafts["player_name_raw"].astype(str).str.strip().eq("") |
    drafts["player_name_raw"].astype(str).str.lower().eq("nan")
)

def draft_match_name(row):
    # DST: try team code first
    if row["pos"] == "DST" and row["nfl_team_code"] in DST_FULL:
        return DST_FULL[row["nfl_team_code"]]
    # DST: try mascot form like "Steelers D/ST"
    if row["pos"] == "DST":
        raw = str(row["player_name_raw"]).strip()
        norm_raw = norm_name(raw)
        # see if “<Mascot> D/ST” is in alias map
        aliased = ALIASES.get(norm_name(raw), None)
        if aliased:
            return aliased
        # if not, keep raw; we’ll still normalize later
        return raw if raw else "D/ST"
    # Non-DST: use parsed name
    return str(row["player_name_raw"]).strip()

drafts["player_name"] = drafts.apply(draft_match_name, axis=1)

# if canonical name ends with D/ST, force pos = DST (ensures correct bucket)
drafts.loc[
    drafts["player_name"].str.contains(r"\bD/ST\b", case=False, na=False),
    "pos"
] = "DST"

# final normalized key
drafts["player_norm"] = drafts["player_name"].map(apply_alias)

# ---------- Normalize ADP & FIN to same key ----------
adps = adps.rename(columns={"Rank":"ADP_Rank"})
for c in ["ADP_Rank","Year"]:
    adps[c] = pd.to_numeric(adps[c], errors="coerce")
adps["player_norm"] = adps["Player"].map(apply_alias)

fin  = fin.rename(columns={"#":"FinishRank","Pos":"POS","TTL":"FantasyPoints"})
for c in ["FinishRank","FantasyPoints","Year"]:
    fin[c] = pd.to_numeric(fin[c], errors="coerce")
fin["player_norm"] = fin["Player"].map(apply_alias)

# ---------- De-duplicate right tables ----------
adps_u = (adps.sort_values(["Year","ADP_Rank"])
              .drop_duplicates(["Year","player_norm"], keep="first"))
fin_u  = (fin.sort_values(["Year","FantasyPoints"], ascending=[True, False])
             .drop_duplicates(["Year","player_norm"], keep="first"))

# ---------- Merge ----------
df = (drafts
      .merge(adps_u[["Year","player_norm","ADP_Rank"]], on=["Year","player_norm"], how="left")
      .merge(fin_u[["Year","player_norm","FinishRank","FantasyPoints"]], on=["Year","player_norm"], how="left")
)

print("Draft rows before merge:", len(drafts))
print("Rows after merge:", len(df))
print("Missing ADP_Rank:", df["ADP_Rank"].isna().sum())
print("Missing FantasyPoints:", df["FantasyPoints"].isna().sum())

# ---------- Diagnostics ----------
# Focus on non-blank rows for “unmatched” counts
valid = df[~df.get("is_blank", False)].copy()
is_dstk = valid["pos"].isin(["DST","K"])
skill   = valid[~is_dstk].copy()

skill["_miss_any"] = skill["ADP_Rank"].isna() | skill["FantasyPoints"].isna()
print("\nTop 30 unmatched (skill positions only):")
print(
    (skill.loc[skill["_miss_any"]]
          .groupby("player_name", dropna=False)
          .size()
          .sort_values(ascending=False)
          .head(30)
          .reset_index(name="missing_rows"))
    .to_string(index=False)
)

print("\nSample DST rows (should be FP city names):")
print(
    valid[valid["pos"]=="DST"][["Year","player_name","ADP_Rank","FinishRank","FantasyPoints"]]
         .sort_values(["player_name","Year"])
         .head(25)
         .to_string(index=False)
)

# quick probes on the recent troublemakers
for nm in ["Patrick Mahomes", "Chris Godwin", "Allen Robinson II", "Ronald Jones II", "AJ Dillon",
           "Aaron Jones", "Kyle Pitts", "Deebo Samuel", "Robby Anderson", "Will Fuller V",
           "Nyheim Hines", "Anthony Richardson"]:
    key = apply_alias(nm)
    print(f"\n--- {nm} (key: {key}) ---")
    print("ADP:",
          adps_u[adps_u["player_norm"]==key][["Year","Player","ADP_Rank"]]
                .sort_values("Year").head(8).to_string(index=False))
    print("FIN:",
          fin_u[fin_u["player_norm"]==key][["Year","Player","FinishRank","FantasyPoints"]]
               .sort_values("Year").head(8).to_string(index=False))
    print("DRF:",
          drafts[drafts["player_norm"]==key][["Year","player_name","pos","OverallPick"]]
                .sort_values("Year").head(8).to_string(index=False))

# clean helper column from df if you prefer
# df = df.drop(columns=["is_blank"], errors="ignore")
# df.to_csv("merged_drafts_adp_fin.csv", index=False)


Draft rows before merge: 1448
Rows after merge: 1448
Missing ADP_Rank: 201
Missing FantasyPoints: 203

Top 30 unmatched (skill positions only):
      player_name  missing_rows
     Chris Godwin             6
         Steelers             6
           Ravens             6
            49ers             5
          Broncos             5
          Cowboys             5
            Bills             5
         Chargers             4
         Patriots             4
          Jaguars             4
         Dolphins             4
             Rams             4
           Saints             4
Melvin Gordon III             3
 Larry Fitzgerald             3
       Buccaneers             3
            Colts             3
            Bears             3
           Texans             3
   Todd Gurley II             3
     Le'Veon Bell             3
          Vikings             3
           Eagles             2
     Duke Johnson             2
   Mark Ingram II             2
           Giants       